This notebook provides a tutorial of bin2cell. (worked with annData extensively)

**bin2cell**: aggregate 2um bins into single cells using DAPI/mCherry/GFP cell segmentation
https://github.com/Teichlab/bin2cell

proposes 2um bin to cell groupings based on segmentation, which can be done on the morphology image and/or a visualisation of the gene expression.  
End result: object with cells, created from grouped 2um bins assigned to the same object after segmentation.  
The following code is looking at their demo notebook: https://nbviewer.org/github/Teichlab/bin2cell/blob/main/notebooks/demo.ipynb

In [2]:
import matplotlib.pyplot as plt
import scanpy as sc
import numpy as np
import os

import bin2cell as b2c

#create directory for stardist input/output files
os.makedirs("stardist", exist_ok=True)

# IDEA: 2um resolution is sub-cellular, can be used for recreating cells

transport.py (219): Blowfish has been deprecated and will be removed in a future release


It starts by correcting for a novel technical effect in the data in variable bin dimensions, and then proposes a bin to cell assignment based on image segmentation. The result is an object with putative cells in it, ready for downstream analysis.   
Data needed: 2um bin output, spatial folder from spaceranger (need to look into), high resolution morphology image (used example dataset in this notebook)

In [8]:
path = "D:/binned_outputs/square_002um"
source_image_path = "D:/Visium_HD_Mouse_Brain_tissue_image.tif"
spaceranger_image_path = "D:/spatial"

In [4]:
# loading count matrix currently needs a bespoke loader function
adata = b2c.read_visium(path, 
                        source_image_path = source_image_path, 
                        spaceranger_image_path = spaceranger_image_path
                       )
adata.var_names_make_unique()
adata

anndata.py (1758): Variable names are not unique. To make them unique, call `.var_names_make_unique`.
anndata.py (1758): Variable names are not unique. To make them unique, call `.var_names_make_unique`.


AnnData object with n_obs × n_vars = 6296688 × 19059
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

Similar issue: data is sparse. Filtering - require gene to show up in 3 spots, require spots to have information

In [5]:
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.filter_cells(adata, min_counts=1)
adata

AnnData object with n_obs × n_vars = 6132629 × 18823
    obs: 'in_tissue', 'array_row', 'array_col', 'n_counts'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
    uns: 'spatial'
    obsm: 'spatial'

bin2cell will perform a segmentation of both the H&E image and a gene expression representation of the data. When performing segmentation, the resolution of the input images is controlled via the mpp parameter.  
mpp - microns per pixel (how many micrometers are captured in each pixel) (?) 

In [6]:
# TODO: what is MPP?
# working with HE image
#likely to be closer to 0.3 for your data
mpp = 0.3
b2c.scaled_he_image(adata, mpp=mpp, save_path="stardist/he.tiff") # image needs to be saved to the drive
# if playing around with mpp, can do store=False

# crops the image to area around the actual spatial grid present in the object, buffer of 150 pixels
# new coordinates: .obsm["spatial_cropped_150_buffer"]
# can plot using sc.pl.spatial()
# upon providing basis="spatial_cropped_150_buffer" and img_key="0.5_mpp_150_buffer" 

Cropped spatial coordinates key: spatial_cropped_150_buffer
Image key: 0.3_mpp_150_buffer


**Working with IF image**  (?)  
use b2c.scaled_if_image(). has a CHANNEL argument - point to index of DAPI signal in the channel list
This creates a greyscale image, which should be segmented with StarDist's fluorescence model with syntax akin to what is done for the GEX representation later in this notebook.

Issue: 2um bins have variability in width and height: will have a striped appearance, with some rows/cols having few transcripts than others.  

In [ ]:
# FIX: identify user-specified quantile (0.99) of total counts, and divide counts of spots in each row by the value. REPEAT for columns
b2c.destripe(adata)

# inspect changes
#define a mask to easily pull out this region of the object in the future
mask = ((adata.obs['array_row'] >= 1450) & 
        (adata.obs['array_row'] <= 1550) & 
        (adata.obs['array_col'] >= 250) & 
        (adata.obs['array_col'] <= 450)
       )

bdata = adata[mask]
sc.pl.spatial(bdata, color=[None, "n_counts", "n_counts_adjusted"], color_map="OrRd",
              img_key="0.3_mpp_150_buffer", basis="spatial_cropped_150_buffer")

# CANNOT PLOT DUE TO MEMORY ISSUES
# adjusted count: multiply the per-spot factor by global quantile of count totals (?)

Can observe a sharper output than the one present in spaceranger (see website)

**H&E Segmentation**

Use StarDist's H&E model. Recommend to lower prob_thresh, image-resolution set by mpp.  
Segmentation results turned into sparse matrix and stored in .npz - read using scipy.sparse.load_npz()  
Sparse matrix's dimensions match the image's.  
Also need to ensure gene expression bin grid is aligned with HE image - misalignment can be fixed by 10X loupe browser.

In [ ]:
b2c.stardist(image_path="stardist/he.tiff", 
             labels_npz_path="stardist/he.npz", # define the path where segmentation label is saved
             stardist_model="2D_versatile_he", 
             prob_thresh=0.01 # probability threshold: A lower threshold (0.01) means StarDist will detect more nuclei, including lower-confidence ones.
            )

# MEMORY FAILURE, but will detect nucleis (?)

Next: load resulting cell calls into the object. (insert labels at each pixel)  
need to inform the function of whether the image is based on array or spatial coordinates

In [ ]:
b2c.insert_labels(adata, 
                  labels_npz_path="stardist/he.npz", # path to the labels generated by stardist
                  basis="spatial", 
                  spatial_key="spatial_cropped_150_buffer", # the key where the cropped image is stored
                  mpp=mpp, 
                  labels_key="labels_he" # where segmentation labels will be stored
                 )

In [ ]:
# visualize
bdata = adata[mask]

#the labels obs are integers, 0 means unassigned
bdata = bdata[bdata.obs['labels_he']>0] # assigned labels
bdata.obs['labels_he'] = bdata.obs['labels_he'].astype(str)

# plot, see on website
sc.pl.spatial(bdata, color=[None, "labels_he"], img_key="0.5_mpp_150_buffer", basis="spatial_cropped_150_buffer") # basis = spatial coordinate basis

In [ ]:
# another way to plot (visualize segmentation)
#the label viewer wants a crop of the processed image
#get the corresponding coordinates spanning the subset object
crop = b2c.get_crop(bdata, basis="spatial", spatial_key="spatial_cropped_150_buffer", mpp=mpp) # retrieve spatial coordinates corresponding to bdata (cropped region)

rendered = b2c.view_labels(image_path="stardist/he.tiff", # overlay segmentation labels
                           labels_npz_path="stardist/he.npz", 
                           crop=crop
                          )
plt.imshow(rendered) # can overlay segmentation region

**Expanded Cell Identification**  
Motivation: previous methods only identify nuclei, while a cell can be bigger

In [ ]:
b2c.expand_labels(adata, 
                  labels_key='labels_he', 
                  expanded_labels_key="labels_he_expanded"
                 )
# finds bins up to 2 bins away from labelled nucleus, joins to corresponding cell
# can capture more bins, as saw on the webpage

# Pro-tip: to customize expansion distance (not just 2 bins), can use algorithm = "volume_ratio" to get a custom distance based on its bin count

**Detecting Missed Cells** (not sure relevance, not explored for now)  Results can be found on the original notebook  
regions that have expression data but lack nuclei to seed a cell, or unusual cell shape.  
FIX: segmentation on representation of total expression per bin (ONLY performs good on sparse tissue, not on dense ones)

**Group bins into cells**  

In [ ]:
cdata = b2c.bin_to_cell(adata, labels_key="labels_joint", spatial_keys=["spatial", "spatial_cropped_150_buffer"])
# features sum of gene expriession in bins, means of array/spatial coords to represent cell centroids
cell_mask = ((cdata.obs['array_row'] >= 1450) & 
             (cdata.obs['array_row'] <= 1550) & 
             (cdata.obs['array_col'] >= 250) & 
             (cdata.obs['array_col'] <= 450)
            )

ddata = cdata[cell_mask]
sc.pl.spatial(ddata, color=["bin_count", "labels_joint_source"], 
              img_key="0.5_mpp_150_buffer", basis="spatial_cropped_150_buffer")

# can also identify source of labels (round 1, or round 2 after detection of missed cells)